In [1]:
from blackbox37 import check
import string
import random
from matplotlib import pyplot as plt
%matplotlib inline

from cross_overs import CrossOverRandomTwo
# from explorations import RandomExploration, SimpleExploration
from mutations import NCharMutation
from population import Population, Individual

ImportError: cannot import name 'generate_population' from 'utils' (C:\Users\pierr\Projets\Polytech\Bio-inspired Intelligence\genetic_algorithms\utils.py)

In [ ]:
available_characters = list(string.ascii_uppercase) + [str(i) for i in range(10)]
available_characters.append("")
GROUP_ID = 2

In [ ]:
print(available_characters)

In [ ]:
def generate():
#     size = random.randint(12, 18)
    size = 18
    individual = random.choices(available_characters, k=size)
    while len(individual) < 12:
        individual = random.choices(available_characters, k=size)
    return individual

In [ ]:
sol = ''.join(generate())
print(sol)
print(check(group_number=2, attempt=sol))

In [ ]:
def plot_results(average, bests):
    plt.figure(figsize=(20, 7))
    plt.plot(average, color='green', linestyle='dotted', label='Moyenne de la population actuelle')
    plt.plot(bests, color='red', label='Maximum trouvé')
    plt.xlabel('Itérations')
    plt.ylabel('Scores')
    plt.legend(loc="upper left")
    plt.show()

# Evolution des populations

In [ ]:
def evolve_population(population, max_epochs=1000, show_progress=False, progress_step=10):
    averages = []
    bests = []
    best = population.population[0]
    for epoch in range(max_epochs):
        population.evaluate()
        b = population.get_max()
        best = b if b.score > best.score else best
        bests.append(best.score)
        average = population.get_average_score()
        averages.append(average)
        
        if show_progress:
            print(f"Epoch {epoch+1}/{epochs}: Best {best.phenotype()} ({best.score}) (average: {average})")
            
        if best.score == 1.0:
            print(f"Best found: {best.phenotype()} at Epoch {epoch+1}/{epochs}!")
            break
            
        pop.evolve()
    return averages, bests, best

## TODO

Rajouter :
- Dans Population
  - get_max() -> Renvoie la classe Individual
  - get_average_score()
- Dans Individual
  - phenotype() -> Retourne le phenotype formaté


In [ ]:
def search(population_size=10, epochs=10):
    cross_over = CrossOverRandomTwo(0.6)
    mutation = NCharMutation(n=2)
    exploration = SimpleExploration(cross_over, mutation, bests_size=int(population_size/10))
    
    population = Population(population_size, exploration=exploration)
    averages, bests, best = evolve_population(population, max_epochs=epochs, show_progress=False)
    print(f"Best solution found: {best.phenotype()} ({best.score})")
    plot_results(averages, bests)

In [ ]:
search()